In [113]:
%pwd

'/Users/dhruv/PROJECTS/Kaggle_Projects/Chest_Cancer_Classification_Project'

In [2]:
import os
os.chdir('../')

In [144]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_type: str
    kaggle_dataset_owner: str
    kaggle_dataset_name: str
    local_data_file: Path
    unzip_dir: Path


In [145]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [146]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,) :
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_type = config.source_type,
            kaggle_dataset_owner = config.kaggle_dataset_owner,
            kaggle_dataset_name = config.kaggle_dataset_name,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config    

In [147]:
import os
import zipfile
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
import shutil

In [155]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self) -> str:
        # Authenticate with Kaggle API
        api = KaggleApi()
        api.authenticate()
        logger.info(f"Kaggle API Authenticated")
        
        try: 
            dataset_url = f"{self.config.kaggle_dataset_owner}/{self.config.kaggle_dataset_name}"
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            api.dataset_download_files(dataset_url, path=zip_download_dir, unzip=False)
            logger.info(f"Downloaded dataset from kaggle to {zip_download_dir}")
            #logger.info(f"Downloaded dataset from kaggle to {self.config.root_dir}")
        
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        zip_file_path = os.path.join(self.config.local_data_file,'chest-ctscan-images.zip')
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)  


        # Delete unnecessary folders (test and valid)
        folders_to_delete = ['test', 'valid']
        for folder in folders_to_delete:
            extract_path = os.path.join(unzip_path, 'Data')
            folder_path = os.path.join(extract_path, folder)
            if os.path.exists(folder_path):
                shutil.rmtree(folder_path)
            else:
                print(f"Folder {folder} does not exist.")
                
        files_to_move_and_rename = [
            {'name': 'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib', 'new_name': 'adenocarcinoma'},
            {'name': 'normal', 'new_name': 'normal'}
        ]  # Update with your file names and new names

        # Loop through the files to move and rename
        for file in files_to_move_and_rename:
            # Get the path to the current file
            file_path = os.path.join(unzip_path, 'Data','train', file['name'])
            
            # Check if the file exists
            if os.path.exists(file_path):
                # Move the file to the zip file path and rename it
                new_file_path = os.path.join(os.path.dirname(self.config.local_data_file), 'Data')
                shutil.move(file_path, new_file_path)
                
            else:
                print(f"File {file['name']} does not exist.")
        shutil.rmtree(os.path.join(unzip_path, 'Data','train')) # Remove the train folder

        
        logger.info(f"Extracted dataset to {unzip_path}")

In [156]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-25 20:32:00,325: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-25 20:32:00,327: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-25 20:32:00,328: INFO: common: created directory at: artifacts]
[2024-08-25 20:32:00,328: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-25 20:32:00,332: INFO: 2990488593: Kaggle API Authenticated]
[2024-08-25 20:32:00,333: INFO: 2990488593: Downloading data from mohamedhanyyy/chest-ctscan-images into file artifacts/data_ingestion/]
Dataset URL: https://www.kaggle.com/datasets/mohamedhanyyy/chest-ctscan-images
[2024-08-25 20:32:02,137: INFO: 2990488593: Downloaded dataset from kaggle to artifacts/data_ingestion/]
[2024-08-25 20:32:02,785: INFO: 2990488593: Extracted dataset to artifacts/data_ingestion/]


In [57]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)
unzip_path =data_ingestion.config.unzip_dir
zip_file_path = os.path.join(data_ingestion.config.local_data_file,'chest-ctscan-images.zip')
#zip_file_path = data_ingestion.config.local_data_file
print({unzip_path},{zip_file_path})


[2024-08-25 16:46:36,508: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-25 16:46:36,510: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-25 16:46:36,511: INFO: common: created directory at: artifacts]
[2024-08-25 16:46:36,512: INFO: common: created directory at: artifacts/data_ingestion]
{'artifacts/data_ingestion/'} {'artifacts/data_ingestion/chest-ctscan-images.zip'}


In [109]:
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(unzip_path) 

# Delete unnecessary folders (test and valid)
folders_to_delete = ['test', 'valid']
for folder in folders_to_delete:
    extract_path = os.path.join(unzip_path, 'Data')
    folder_path = os.path.join(extract_path, folder)
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
    else:
        print(f"Folder {folder} does not exist.")
        
files_to_move_and_rename = [
    {'name': 'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib', 'new_name': 'adenocarcinoma'},
    {'name': 'normal', 'new_name': 'normal'}
]  # Update with your file names and new names

# Loop through the files to move and rename
for file in files_to_move_and_rename:
    # Get the path to the current file
    file_path = os.path.join(unzip_path, 'Data','train', file['name'])
    
    # Check if the file exists
    if os.path.exists(file_path):
        # Move the file to the zip file path and rename it
        new_file_path = os.path.join(os.path.dirname(zip_file_path), 'Data')
        shutil.move(file_path, new_file_path)
        
    else:
        print(f"File {file['name']} does not exist.")
shutil.rmtree(os.path.join(unzip_path, 'Data','train'))


In [106]:
# Define the files to move and rename in the Data folder
os.rename(folder_path, new_folder_path)
# Print a success message
print({file_path},{new_file_path})

{'artifacts/data_ingestion/Data/train/normal'} {'artifacts/data_ingestion/Data'}


In [108]:
shutil.rmtree(os.path.join(unzip_path, 'Data','train'))

In [94]:
extract_path = os.path.join(unzip_path, 'Data','train')
extract_path

'artifacts/data_ingestion/Data/train'

In [76]:
# Define the files to move and rename in the Data folder
files_to_move_and_rename = [
    {'name': 'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib', 'new_name': 'adenocarcinoma'},
    {'name': 'normal', 'new_name': 'normal'}
]  # Update with your file names and new names

# Loop through the folders to delete and move
for folder in files_to_move_and_rename:
    # Get the path to the current folder
    folder_path = os.path.join(unzip_path, 'Data', folder['name'])
    
    # Check if the folder exists
    if os.path.exists(folder_path):
        # Move the folder to the zip file path and rename it
        new_folder_path = os.path.join(new_folder_path,'Data',os.path.dirname(folder['new_name']))
        shutil.move(folder_path, new_folder_path)
    else:
        print(f"Folder {folder['name']} does not exist.")

# Print a success message
print("Unzipped, deleted and moved folders successfully.")

Unzipped, deleted and moved folders successfully.


In [72]:
#folders_to_delete = ['large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa', 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa']
files_to_move_and_rename = [
    {'name': 'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib', 'new_name': 'adenocarcinoma'},
    {'name': 'normal', 'new_name': 'normal'}
]
folder_path = os.path.join(unzip_path, 'Data', 'train', folder['name'])
folder_path
new_folder_path = os.path.join(new_folder_path,'Data',os.path.dirname(folder['new_name']))
new_folder_path
print({folder_path},{new_folder_path})

{'artifacts/data_ingestion/Data/train/normal'} {'artifacts/data_ingestion/normal/Data/'}


In [71]:
new_folder_path = os.path.join(os.path.dirname(zip_file_path), folder['new_name'])
new_folder_path

'artifacts/data_ingestion/normal'

In [23]:
folder_path

'artifacts/data_ingestion/Data/train'

In [ ]:
extract_path = os.path.join(unzip_path,'Data')
folder_path = os.path.join(extract_path, folder)
folder_path

In [ ]:
shutil.move(os.path.join(extract_path, 'train'), extract_path)